# Collecting (month, average players, peak players) per game per month each year
Takes in an array of arrays that looks like: [ [rank, app_id], [rank, app_id], ... ]
Outputs: [ [app_id, month, avg_players, peak_players], ... ] 

In [6]:
# imports and globals
import csv
import time
import hashlib
from pathlib import Path 
from typing import Optional

import requests
import pandas as pd
from bs4 import BeautifulSoup # for parsing HTML docs

# Base URL pattern for SteamCharts game pages
# inject app_id into {app_id}, e.g. app_id=730 -> https://steamcharts.com/app/730
BASE_URL = "https://steamcharts.com/app/{appid}"

# Apprently some sites block requests that do not provide a browser-like user agent.
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; SteamChartsYearScraper/1.0)"
}

In [2]:
# Some utlity helpers! ==========================================================================

def clean_num(value: str) -> Optional[float]:
    """
    Convert numeric text to float.

    Handles:
    - commas: "12,345.6" -> 12345.6
    - blanks/dashes -> None
    - invalid values -> None
    """
    if value is None:
        return None

    text = str(value).strip().replace(",", "")
    if text in {"", "-", "—"}:
        return None

    try:
        return float(text)
    except ValueError:
        return None


def build_game_url(appid: int) -> str:
    """
    Build SteamCharts URL for one appid.
    Example: appid=730 -> "https://steamcharts.com/app/730"
    """
    return BASE_URL.format(appid=int(appid))



def cache_key_for_url(url: str) -> str:
    """
    Build deterministic cache filename key from URL.
    Using md5 keeps filenames short and filesystem-safe.
    Why this exists:
    - URL text may not be ideal as a filename.
    - Hash gives stable and filesystem-safe names.
    """
    return hashlib.md5(url.encode("utf-8")).hexdigest()



# Input and loading validation ==================================================================


def load_input_csv(file_path: Path) -> pd.DataFrame:
    """
    Load and validate one input CSV.
    (Probably not neccessary, but I think it's good practice just in case)

    Required columns:
    - rank
    - name
    - appid

    Returns:
    - Cleaned DataFrame with normalized dtypes:
      rank:int, name:str, appid:int

    Why this exists:
    - Fail fast with clear errors if schema is wrong.
    - Avoid repeated type conversions elsewhere.
    """
    # TODO:
    # Read CSV (consider encoding="utf-8-sig")
    df = pd.read_csv(file_path, encoding="utf-8-sig")

    # Validate required columns
    required = {"rank", "name", "appid"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(
            f"{file_path.name} is missing required columns: {sorted(missing)}. "
            f"Found columns: {list(df.columns)}"
        )

    # Keep only needed columns in predictable order 
    df = df[["rank", "name", "appid"]].copy()

    # Convert numeric fields and fail LOUDLY if invalid
    df["rank"] = pd.to_numeric(df["rank"], errors="raise").astype(int)
    df["appid"] = pd.to_numeric(df["appid"], errors="raise").astype(int)

    # 4) Strip whitespace on name
    df["name"] = df["name"].astype(str).str.strip()

    # Remove rows with empty names 
    df = df[df["name"] != ""].copy()

    # drop duplicates on rank+appid
    df = df.drop_duplicates(subset=["rank", "appid"]).reset_index(drop=True)

    # Return cleaned DataFrame
    # Sort by rank for deterministic processing
    df = df.sort_values("rank").reset_index(drop=True)


    return df


# Network + cache =================================================================================


def get_game_page_html(
    appid: int,
    session: requests.Session,
    cache_dir: Path,
    use_cache: bool = True,
    request_delay_sec: float = 0.6,
) -> str:
    """
    Return HTML for one game page, using cache when available.

    Flow:
    1) Build game URL from appid
    2) Compute cache filename from URL hash
    3) If cache exists and use_cache=True -> return cached HTML
    4) Else fetch from network, save cache, sleep briefly, return HTML
    """
    # TODO:
    # 1) Ensure cache_dir exists
    cache_dir.mkdir(parents=True, exist_ok=True)

    # 2) Compute URL and cache filename
    url = build_game_url(appid)
    cache_file = cache_dir / f"{cache_key_for_url(url)}.html"

    # 3) If cache hit and use_cache: read + return
    if cache_file.exists() and use_cache:
        return cache_file.read_text(encoding="utf-8")

    # Live request path
    resp = session.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    html = resp.text

    # Save to cache for future runs
    cache_file.write_text(html, encoding="utf-8")

    # Don't attack our lord and savior GabeN with rapid-fire requests
    time.sleep(request_delay_sec)

    return html


# HTML parsing ===================================================================================


def parse_year_data_from_html(html: str, target_year: int) -> list[dict]:
    """
    Parse SteamCharts monthly table from one app page, filtered to target year.

    Input:
    - html: raw page HTML
    - target_year: year to keep (e.g., 2021)

    Output row shape:
    {
      "month": "YYYY-MM",
      "avg_players": float|None,
      "peak_players": float|None
    }

    Why this exists:
    - Pure parser function (HTML in -> structured rows out).
    - Easy to test independently from I/O.
    """
    # TODO:
    # 1) Parse html with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    parsed_rows = []

    # 2) Select monthly table rows (table.common-table tbody tr)
    rows = soup.select("table.common-table tbody tr")

    # 3) For each row, parse month/avg/peak
    for tr in rows: 
        tds = tr.find_all("td")

        # Monthly rows should have at least 5 columns
        # Month | Avg. Players | Gain | Gain % | Peak Players
        if len(tds) < 5:
            continue

        month_text = tds[0].get_text(" ", strip=True)

        # 4) Skip "Last 30 Days"
        if month_text.lower() == "last 30 days":
            continue

        # 5) Parse month text with pd.to_datetime(..., format="%B %Y")
        month_dt = pd.to_datetime(month_text, format="%B %Y", errors="coerce")
        if pd.isna(month_dt):
            continue

        # 6) Keep rows where parsed year == target_year
        if int(month_dt.year) != int(target_year):
            continue

        # 7) Clean numeric fields with clean_num()
        avg_text = tds[1].get_text(" ", strip=True)
        peak_text = tds[4].get_text(" ", strip=True)

        # 8) Return rows sorted by month asc
        parsed_rows.append({
            "month": month_dt.strftime("%Y-%m"),
            "avg_players": clean_num(avg_text),
            "peak_players": clean_num(peak_text),
        })

    # Keep output in chronological order
    parsed_rows.sort(key=lambda r: r["month"])
    return parsed_rows



# year collection ====================================================================================


def collect_one_year(
    input_csv: Path,
    year: int,
    cache_dir: Path,
    use_cache: bool = True,
    request_delay_sec: float = 0.6,
) -> pd.DataFrame:
    """
    Collect SteamCharts data for one year's input list.

    Steps:
    - Load CSV (rank, name, appid)
    - For each game:
      - Fetch or read cached HTML
      - Parse target-year monthly rows
      - Emit result rows with status labels

    Status values:
    - "ok"               : parsed monthly rows exist
    - "no_data_for_year" : page loaded, but no rows for that year
    - "request_error"    : failed HTTP request
    - "parse_error"      : page fetched but parse failed
    """
    # TODO:
    # 1) games_df = load_input_csv(input_csv)
    games_df = load_input_csv(input_csv)

    # 2) Initialize out_rows = []
    out_rows = []
    total = len(games_df)

    # 3) Create requests.Session()
    with requests.Session() as session:
        for idx, row in games_df.iterrows():
            rank = int(row["rank"])
            name = row["name"]
            appid = int(row["appid"])

            # Fetch HTML (cache first)
            try: 
                html = get_game_page_html(
                    appid=appid,
                    session=session,
                    cache_dir=cache_dir,
                    use_cache=use_cache,
                    request_delay_sec=request_delay_sec,
                )
            except Exception:
                out_rows.append({
                    "year": year,
                    "rank": rank,
                    "name": name,
                    "appid": appid,
                    "month": None,
                    "avg_players": None,
                    "peak_players": None,
                    "status": "request_error",
                })
                print(f"[{idx+1}/{total}] {name} (appid={appid}): request error")
                continue

            # Parse only rows for target year
            try:
                parsed_rows = parse_year_data_from_html(html, target_year=year)
            except Exception:
                out_rows.append({
                    "year": year,
                    "rank": rank,
                    "name": name,
                    "appid": appid,
                    "month": None,
                    "avg_players": None,
                    "peak_players": None,
                    "status": "parse_error",
                })
                print(f"[{idx+1}/{total}] {name} (appid={appid}): parse error")
                continue

            # no rows found for this yeaer 
            if not parsed_rows:
                out_rows.append({
                    "year": year,
                    "rank": rank,
                    "name": name,
                    "appid": appid,
                    "month": None,
                    "avg_players": None,
                    "peak_players": None,
                    "status": "no_data_for_year",
                })
                print(f"[{idx+1}/{total}] {name} (appid={appid}): no data for year")
                continue

            # Found rows: attach metadata 
            for pr in parsed_rows:
                out_rows.append({
                    "year": year,
                    "rank": rank,
                    "name": name,
                    "appid": appid,
                    "month": pr["month"],
                    "avg_players": pr["avg_players"],
                    "peak_players": pr["peak_players"],
                    "status": "ok",
                })

            print(f"[{idx+1}/{total}] {name} ({appid}) -> ok ({len(parsed_rows)} months)")

    result_df = pd.DataFrame(
        out_rows,
        columns=[
            "year", 
            "rank", 
            "name", 
            "appid",
            "month", 
            "avg_players", 
            "peak_players",
            "status",
        ],)

    # Sort for readability (rank, then month)
    result_df = result_df.sort_values(
                    by=["rank", "month"], 
                    na_position="last"
                ).reset_index(drop=True)

    return result_df


def collect_year_range(
    start_year: int,
    end_year: int,
    input_dir: Path,
    input_pattern: str,      # e.g. "{year}_top250_ids.csv"
    output_dir: Path,
    cache_dir: Path,
    use_cache: bool = True,
    request_delay_sec: float = 0.6,
    write_combined: bool = True,
) -> pd.DataFrame:
    """
    Run collection across a year range using predictable filenames.

    For each year:
    - Build input file path from input_pattern
    - Skip year if file missing
    - Collect year data
    - Write per-year CSV

    Optionally:
    - Combine all years into one DataFrame + CSV
    """
    # TODO:
    # Ensure output_dir exists
    output_dir.mkdir(parents=True, exist_ok=True)
    cache_dir.mkdir(parents=True, exist_ok=True)

    all_parts = []

    for year in range(start_year, end_year + 1):
    #   build input_csv path from pattern
        input_csv = input_dir / input_pattern.format(year=year)

    #   if missing file: print skip and continue
        if not input_csv.exists():
            print(f"{year} SKIP - missing input file:{input_csv}")
            continue

        year_df = collect_one_year(
            input_csv=input_csv,
            year=year,
            cache_dir=cache_dir,
            use_cache=use_cache,
            request_delay_sec=request_delay_sec,
        )
    #   write year_df to output_dir / f"steamcharts_{year}_top250.csv"
        year_out_path = output_dir / f"steamcharts_{year}_top250.csv"
        year_df.to_csv(year_out_path, index=False)
        print(f"[{year}] wrote {year_out_path} ({len(year_df)} rows)")

        status_counts = year_df["status"].value_counts(dropna=False)
        print(f"[{year}] status summary:\n{status_counts.to_string()}")

    #   append year_df to all_parts
        all_parts.append(year_df)

    # If nothing processed, return empty with expected schema
    if not all_parts:
        return pd.DataFrame(columns=[
            "year", "rank", "name", "appid", "month",
            "avg_players", "peak_players", "status"
        ])

    combined_df = pd.concat(all_parts, ignore_index=True)

    if write_combined:
        combined_out_path = output_dir / f"steamcharts_{start_year}_{end_year}_combined.csv"
        combined_df.to_csv(combined_out_path, index=False)
        print(f"\nWrote combined file: {combined_out_path} ({len(combined_df)} rows)")

    return combined_df

# TESTING ===================================================================================================
def _test_load_input_csv():
    test_path = Path("2018_top250_ids.csv")
    df = load_input_csv(test_path)

    print("Loaded rows:", len(df))
    print("Columns:", df.columns.tolist())
    print(df.head(10).to_string(index=False))


def _test_fetch_html():
    cache_dir = Path("steamcharts_cache_test")
    appid = 730  # CS

    with requests.Session() as session:
        # First call should fetch live and cache
        html1 = get_game_page_html(
            appid=appid,
            session=session,
            cache_dir=cache_dir,
            use_cache=True,
            request_delay_sec=0.6,
        )
        print("First fetch length:", len(html1))

        # Second call should come from cache (no live request needed)
        html2 = get_game_page_html(
            appid=appid,
            session=session,
            cache_dir=cache_dir,
            use_cache=True,
            request_delay_sec=0.6,
        )
        print("Second fetch length:", len(html2))

    print("Same content:", html1 == html2)
    print("Cache files:", len(list(cache_dir.glob("*.html"))))


def _test_parse_html():
    appid = 730
    target_year = 2021
    cache_dir = Path("steamcharts_cache_test")

    with requests.Session() as session:
        html = get_game_page_html(
            appid=appid,
            session=session,
            cache_dir=cache_dir,
            use_cache=True,          # should hit cache if Step 2 already ran
            request_delay_sec=0.6,
        )

    rows = parse_year_data_from_html(html, target_year=target_year)

    print(f"Parsed rows for appid={appid}, year={target_year}: {len(rows)}")
    print("Preview:")
    for r in rows[:12]:
        print(r)

    
def _test_collect_one_year():
    input_csv = Path("2018_top250_ids.csv")
    year = 2018
    cache_dir = Path("steamcharts_cache_test")

    df = collect_one_year(
        input_csv=input_csv,
        year=year,
        cache_dir=cache_dir,
        use_cache=True,
        request_delay_sec=0.6,
    )

    print("\nResult shape:", df.shape)
    print(df.head(20).to_string(index=False))

    print("\nStatus counts:")
    print(df["status"].value_counts(dropna=False).to_string())

# MAIN =======================================================================================================


def main():
    """
    Central run configuration.

    Keep all user-editable settings here so the rest of the code
    stays stable and easy to reason about.
    """

    start_year = 2018
    end_year = 2023

    # Where files are located (~/data/00-raw/)
    input_dir = Path(".")

    # File name pattern for input CSVs, with {year} placeholder, e.g. "2018_top250_ids.csv"
    input_pattern = "{year}_top250_ids.csv"

    # Output and cache directory for per-year and combined CSVs
    output_dir = Path("outputs")
    cache_dir = Path("steamcharts_cache")

    # Cache behavior: True = reuse cached pages if present
    use_cache = True

    # Delay for live requests (in seconds) to avoid hammering the server
    request_delay_sec = 0.6

    combined_df = collect_year_range(
        start_year=start_year,
        end_year=end_year,
        input_dir=input_dir,
        input_pattern=input_pattern,
        output_dir=output_dir,
        cache_dir=cache_dir,
        use_cache=use_cache,
        request_delay_sec=request_delay_sec,
        write_combined=True,
    )

    print("\nCombined preview:")
    print(combined_df.head(20).to_string(index=False))

    print("\nCombined status summary:")
    if not combined_df.empty:
        print(combined_df["status"].value_counts(dropna=False).to_string())
    else:
        print("No data collected. Check input file paths/pattern.")


if __name__ == "__main__":
    main()

[1/250] Rust (252490) -> ok (12 months)
[2/250] Among Us (945360) -> ok (2 months)
[3/250] The Forest (242760) -> ok (12 months)
[4/250] Monster Hunter: World (582010) -> ok (5 months)
[5/250] DayZ (221100) -> ok (12 months)
[6/250] Bloons TD 6 (960090) -> ok (1 months)
[7/250] Paladins® (444090) -> ok (12 months)
[8/250] Subnautica (264710) -> ok (12 months)
[9/250] Battlefield™ V (appid=1238810): no data for year
[10/250] RimWorld (294100) -> ok (12 months)
[11/250] Z1 Battle Royale (433850) -> ok (12 months)
[12/250] Far Cry® 5 (552520) -> ok (9 months)
[13/250] Assassin's Creed® Odyssey (812140) -> ok (3 months)
[14/250] Dead Cells (588650) -> ok (12 months)
[15/250] Kingdom Come: Deliverance (379430) -> ok (11 months)
[16/250] Warhammer: Vermintide 2 (552500) -> ok (12 months)
[17/250] DARK SOULS™: REMASTERED (570940) -> ok (8 months)
[18/250] Celeste (504230) -> ok (12 months)
[19/250] Frostpunk (323190) -> ok (12 months)
[20/250] Insurgency: Sandstorm (581320) -> ok (1 months)
[

In [ ]:
# CHANGE THIS PATH if needed
combined_path = Path("outputs/steamcharts_2018_2023_combined.csv")

df = pd.read_csv(combined_path)

# Columns
print("=== Columns ===")
print(df.columns.tolist())

# Overall size
print("\n=== Overall Size ===")
print(f"Rows: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")

# Status counts
print("\n=== Status Counts ===")
if "status" in df.columns:
    print(df["status"].value_counts(dropna=False).to_string())
else:
    print("No 'status' column found.")

# Period-level summary (descriptive only)
required = {"year", "avg_players", "peak_players"}
if required.issubset(df.columns):
    tmp = df.copy()

    # light coercion for summary calculations only
    tmp["year"] = pd.to_numeric(tmp["year"], errors="coerce")
    tmp["avg_players"] = pd.to_numeric(tmp["avg_players"], errors="coerce")
    tmp["peak_players"] = pd.to_numeric(tmp["peak_players"], errors="coerce")

    # use only successful scrape rows if status exists
    if "status" in tmp.columns:
        tmp = tmp[tmp["status"] == "ok"].copy()

    def period_label(y):
        if pd.isna(y):
            return "unknown"
        y = int(y)
        if 2018 <= y <= 2019:
            return "pre_covid"
        elif 2020 <= y <= 2021:
            return "covid"
        elif 2022 <= y <= 2023:
            return "post_covid"
        return "other"

    tmp["period"] = tmp["year"].apply(period_label)

    period_summary = (
        tmp.groupby("period", as_index=False)
           .agg(
               n_rows=("period", "size"),
               avg_players_mean=("avg_players", "mean"),
               avg_players_median=("avg_players", "median"),
               peak_players_mean=("peak_players", "mean"),
               peak_players_median=("peak_players", "median"),
           )
           .sort_values("period")
    )

    print("\n=== Period-Level Summary (status='ok') ===")
    print(period_summary.to_string(index=False))
else:
    print("\nMissing one or more required columns for period summary:", required)

=== Columns ===
['year', 'rank', 'name', 'appid', 'month', 'avg_players', 'peak_players', 'status']

=== Overall Size ===
Rows: 10444
Columns: 8

=== Status Counts ===
status
ok                  10395
request_error          37
no_data_for_year       12

=== Period-Level Summary (status='ok') ===
    period  n_rows  avg_players_mean  avg_players_median  peak_players_mean  peak_players_median
     covid    3337       1925.576961              256.13        4481.250524                648.0
post_covid    3575       3925.858859              341.25        8455.939860                838.0
 pre_covid    3483       1367.490215              127.23        3135.848119                347.0
